In [14]:
import molsysmt as msm
import numpy as np

In [2]:
molsys_1 = msm.convert(msm.demo['T4 lysozyme L99A']['181l.msmpk'], to_form='molsysmt.MolSys')

In [3]:
molsys_1 = msm.extract(molsys_1, selection='molecule_type=="protein"')

In [4]:
molsys_2 = msm.convert(msm.demo['T4 lysozyme L99A']['1l17.msmpk'], to_form='molsysmt.MolSys')

In [5]:
molsys_2 = msm.convert(molsys_2, to_form='molsysmt.MolSys', selection='molecule_type=="protein"')

In [6]:
molsys_2on1 = msm.structure.align(molsys_2, selection='backbone',
                                      reference_molecular_system=molsys_1,
                                      reference_selection='backbone')

In [7]:
identity, int2, int1 = msm.topology.get_sequence_identity(molsys_2, reference_molecular_system=molsys_1)

In [8]:
aux1 = msm.select(molsys_1, selection='group_index==@int1')

In [9]:
sel1 = msm.select(molsys_1, selection='backbone', mask=aux1)

In [10]:
aux2 = msm.select(molsys_2, selection='group_index==@int2')

In [11]:
sel2 = msm.select(molsys_2, selection='backbone', mask=aux2)

In [12]:
rmsd = msm.structure.get_rmsd(molsys_2on1, selection=sel2, reference_molecular_system=molsys_1, reference_selection=sel1)

In [15]:
check = np.isclose(0.021332505947, msm.pyunitwizard.get_value(rmsd, to_unit='nm')[0])

In [16]:
check

False

In [17]:
msm.pyunitwizard.get_value(rmsd, to_unit='nm')[0]

0.018528272410565524

In [ ]:
check = np.isclose(0.021332505947, msm.pyunitwizard.get_value(rmsd, to_unit='nm')[0])
print(msm.pyunitwizard.get_value(rmsd, to_unit='nm')[0])

assert check
